In [1]:
# Technical libraries
import cvxpy as cp
import numpy as np
import numpy.matlib
import numpy.random
from scipy import linalg, special
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy 
import scipy.fftpack
# from proximal import ProxUtils
import proximal
from iplabs import IPLabViewer as viewer
import matplotlib
%matplotlib widget

# Proximal Operators for Nonnegative Inverse Problems

Study of the combination of different image regularizers with nonnegativity constraints. This notebook is dedicated to evaluations only. See file [`proximal.py`](proximal.py) for details on the functions.

# Index

1. [Introduction](#Intro)
    1. [Proximal Operator](#Prox)
    2. [Nonnegativity Function](#Nonneg)
2. [Norms](#2.-Norms)
    1. [$\ell_p$](#2.A-Lp)
    2. [$\ell_p^p$](#2.B-Lp^p)
3. [Group Sparsity](#3.-Group-Sparsity)
    1. [1D Group Sparsity](#3.A-1D-Group-Sparsity)
    2. [2D Group Sparsity](#3.B-2D-Group-Sparsity)
4. [Total Variation](#4.-Spectral-Norm)
    1. [1D Total Variation](#4.A-1D-Total-Variation)
    2. [2D Total Variation](#4.B-2D-Total-Variation)
5. [Schatten Norm](#5.-Schatten-Norm)
6. [Hessian Schatten](#6.-Hessian-Schatten-Norm)

# <a name="Intro"></a>1. Introduction

The goal of the present notebook is to find how of common image regularizers combine with nonnegativity through the proximal operator. 

## <a name="Prox"></a>1.A. Proximal Operator

The proximal operator of a function $f$ is:

$$\mathrm{prox}_f(v) = \arg \min_x(f(x)+\frac{1}{2\lambda}||x - v||_2^2)$$

This means that it will optimize an input vector $v$ with respect to a function, but adding the constraint that the result has to be *somewhat close* (by the minimization of the second term, and with *somewhat* parametrized by $\lambda$) to the original. 

The interest of the project is to see how common image regularizers combine with nonnegativity constraints. In particular, we will see wether:
$$\mathrm{prox}_{f} = \mathrm{prox}_{\delta_{\rm I\!R_+^N}}(\mathrm{prox}_{\Re})$$
or 
$$\mathrm{prox}_{f} = \mathrm{prox}_{\Re}(\mathrm{prox}_{\delta_{\rm I\!R_+^N}})$$
where $f = \delta_{\rm I\!R_+^N} + \Re$, for several regularizers. Therefore we have $3$ cases to solve, the left-hand-side of both equations (which is the same and corresponds to the ground truth), and the $2$ right-hand-sides.

## <a name="Nonneg"></a>1.B. Nonnegativity Function

The nonnegativity function $\delta$ is defined as:

$$\delta_{\rm I\!R_+^N} =
\begin{cases}
        0 \mathrm{ if } x \in \rm I\!R_+^N \\
        + \inf \mathrm{ if } x\notin \rm I\!R_+^N
     \end{cases}$$
     
Given the definition of the proximal operator, it is immediate to see that to solve the proximal operator, we have as constraint $x\in \rm I\!R_+^N$. As such, the first term vanishes (since $x\in \rm I\!R_+^N$, by definition $f(x) = 0$). As such, we arrive to:

$$prox_{\rm I\!R_+^N}(v) = \arg \min_{x\in \rm I\!R_+^N}\left(\frac{1}{2}||x - v||_2^2\right)$$

Given that $v \in \rm I\!R_+^N$, then the $x\in \rm I\!R_+^N$ that minimizes the prox operator is simply the $x$ closest to $v$, but yet inside the domain imposed by the indicator function (indicator of nonnegativity). It is useless to include the parameter $\lambda$ in this proximal, as the minimization of squares is the only term present. The result of the previous equation is that any negative element will be mapped to $0$, and the rest of the elements will be left untouched. 

# 2. Norms
[Back to Index](#Index)

In this section we will prove the equations provided on [1.A](#Prox), for the $\mathrm{L}^1$ norm. It is a part from the family of the $L^p$ norms where $p = 1$, is defined for a vector $x$ as:
$$\|\mathbf{x}\|_p = (\sum_{i = 1}^n |x|_n^p)^\frac{1}{p} $$

### 2.A Lp
[Back to Index](#Index)

In [2]:
for p in [1, 2, 3, 5, 8, 10]:
    print(f'||p norm where p = {p}')
    if p in [2]:
        proximal.evaluate(proximal.Lp_prox, proximal.Lp_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, additional_params=[p], err_thr=1e-3, plot=True)
        continue
    proximal.evaluate(proximal.Lp_prox, proximal.Lp_plus_nonneg_prox, n=100, size=(30, ), mean=0, sigma=2, lamb=0.5, plot=False, additional_params=[p], err_thr = 1e-2)


||p norm where p = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 3.348e-04
Average absolute error: 3.316e-05

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 5.237e-04
Average absolute error: 4.774e-06

||p norm where p = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 8.975e-02
Average absolute error: 1.263e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 9.989e-05
Average absolute error: 3.800e-06

Plotting example


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p norm where p = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.494e-01
Average absolute error: 1.142e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 2.943e-04
Average absolute error: 3.817e-06

||p norm where p = 5
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.467e-01
Average absolute error: 9.999e-03

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 5.454e-04
Average absolute error: 3.652e-06

||p norm where p = 8
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 4.052e-01
Average absolute error: 9.372e-03

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 3.930e-04
Average absolute error: 3.495e-06

||p norm where p = 10
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 4.475e-01
Average absolute error: 8.965e-03

prox_reg(prox_nonneg(v)) SEEMS equal to pro

### 2.B Lp^p
[Back to Index](#Index)

$$\|\mathbf{x}\|_p = (\sum_{i = 1}^n |x_n|^p) $$

In [3]:
for p in [1, 2, 3, 5, 8]:
    print(f'||p^p norm where p = {p}')
    if p == 5:
        proximal.evaluate(proximal.Lp_prox, proximal.Lp_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=2, lamb=0.5, additional_params=[p, True], err_thr=1e-3, plot=True)
        continue
    proximal.evaluate(proximal.Lp_prox, proximal.Lp_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=2, lamb=0.5, plot=False, additional_params=[p, True], err_thr = 1e-3)

||p^p norm where p = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 3.793e-04
Average absolute error: 3.205e-05

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.225e-04
Average absolute error: 4.811e-06

||p^p norm where p = 2
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.141e-04
Average absolute error: 1.069e-05

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 7.211e-05
Average absolute error: 8.635e-06

||p^p norm where p = 3
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.503e-04
Average absolute error: 4.439e-06

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 4.280e-05
Average absolute error: 8.267e-07

||p^p norm where p = 5
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.235e-04
Average absolute error: 5.830e-06

prox_reg(prox_nonneg(v)) SEEMS equal

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p^p norm where p = 8
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 7.972e-05
Average absolute error: 3.041e-06

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.577e-04
Average absolute error: 1.510e-06



## 3. Group Sparsity
[Back to Index](#Index)

This section will calculate the equations of interest for Group Sparsity. The expression is the same for the $1$-dimensional and $2$-dimensional cases, and is given by
$$\|\mathbf{x}\|_{p, q} = \left(\sum^S_{i = 1}\|\mathbf{x}_i\|_p^q\right)^\frac{1}{q}$$.

### 3.A 1D Group Sparsity

In [4]:
for p in [1, 2, 3]:
    for q in [1, 2, 3]:
        print(f'||p norm where p = {p}, q = {q}')
        if p == 2 and q == 2:
            proximal.evaluate(proximal.GS_1D_prox, proximal.GS_1D_plus_nonneg_prox, n=10, size=(60, ), mean=0, sigma=1, lamb=0.5, additional_params=[6, p, q], err_thr=1e-3, plot=True)
            continue
#         else:
#             continue
        proximal.evaluate(proximal.GS_1D_prox, proximal.GS_1D_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[5, p, q], err_thr = 1e-3)

||p norm where p = 1, q = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.390e-04
Average absolute error: 3.581e-05

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 5.005e-05
Average absolute error: 1.792e-06

||p norm where p = 1, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.755e-01
Average absolute error: 2.097e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.129e-04
Average absolute error: 4.182e-06

||p norm where p = 1, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.712e-01
Average absolute error: 3.799e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 2.663e-05
Average absolute error: 1.551e-06

||p norm where p = 2, q = 1
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.589e-01
Average absolute error: 3.355e-02

prox_reg(prox_no

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p norm where p = 2, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.889e-01
Average absolute error: 1.112e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 7.510e-05
Average absolute error: 3.812e-06

||p norm where p = 3, q = 1
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 4.449e-01
Average absolute error: 3.866e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 7.636e-05
Average absolute error: 1.651e-06

||p norm where p = 3, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.527e-01
Average absolute error: 1.556e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 3.890e-05
Average absolute error: 1.311e-06

||p norm where p = 3, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.317e-01
Average absolute error: 1.308e-02

prox_reg(prox_no

$$\|\mathbf{x}\|_{p, q}^{p, q} = \sum^S_{i = 1}\left(\|\mathbf{x}_i\|_p^p\right)^q$$.

In [5]:
for p in [1, 2, 3]:
    for q in [1, 2, 3]:
        print(f'||p norm where p = {p}, q = {q}')
        if p == 1 and q == 2:
            proximal.evaluate(proximal.GS_1D_prox, proximal.GS_1D_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, additional_params=[5, p, q, True], err_thr=1e-3, plot=True)
            continue
        proximal.evaluate(proximal.GS_1D_prox, proximal.GS_1D_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[5, p, q, True], err_thr = 1e-3)

||p norm where p = 1, q = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.333e-04
Average absolute error: 3.479e-05

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 5.991e-05
Average absolute error: 1.895e-06

||p norm where p = 1, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.161e-01
Average absolute error: 2.994e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 5.600e-05
Average absolute error: 1.810e-06

Plotting example


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p norm where p = 1, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 7.034e-01
Average absolute error: 4.218e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 2.050e-05
Average absolute error: 1.304e-06

||p norm where p = 2, q = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 8.073e-05
Average absolute error: 2.926e-06

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 9.715e-05
Average absolute error: 2.795e-06

||p norm where p = 2, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.683e-01
Average absolute error: 2.810e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 5.621e-05
Average absolute error: 2.844e-06

||p norm where p = 2, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.767e-01
Average absolute error: 4.070e-02

prox_reg(prox_no

### 3.B 2D Group Sparsity
[Back to Index](#Index)

As aforementioned, the expressions are the same as for the $1$-dimensional cases. In the next cell I test for 
$$\|\mathbf{x}\|_{p, q} = \left(\sum^S_{i = 1}\|\mathbf{x}_i\|_p^q\right)^\frac{1}{q}$$.

In [6]:
for p in [1, 2]:
    for q in [1, 2, 3]:
        print(f'||p norm where p = {p}, q = {q}')
        if p == 1 and q == 2:
            proximal.evaluate(proximal.GS_prox, proximal.GS_plus_nonneg_prox, n=10, size=(40, 40,), mean=0, sigma=1, lamb=0.5, additional_params=[(4, 4), p, q], err_thr=1e-3, plot=True)
            continue
        proximal.evaluate(proximal.GS_prox, proximal.GS_plus_nonneg_prox, n=10, size=(40, 40,), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[(4, 4), p, q], err_thr = 1e-3)

||p norm where p = 1, q = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 3.067e-04
Average absolute error: 4.776e-06

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 2.116e-04
Average absolute error: 2.233e-07

||p norm where p = 1, q = 2
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 4.571e-04
Average absolute error: 5.063e-06

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 3.337e-04
Average absolute error: 3.727e-07

Plotting example as image.


Button(description='Show Widgets', style=ButtonStyle())

||p norm where p = 1, q = 3
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 3.314e-04
Average absolute error: 3.956e-06

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 3.158e-04
Average absolute error: 2.744e-07

||p norm where p = 2, q = 1
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 7.451e-02
Average absolute error: 5.429e-04

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.173e-04
Average absolute error: 1.630e-07

||p norm where p = 2, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 7.508e-02
Average absolute error: 5.519e-04

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 2.079e-04
Average absolute error: 2.139e-07

||p norm where p = 2, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.745e-02
Average absolute error: 5.177e-04

prox_reg(prox_no

$$\|\mathbf{x}\|_{p, q}^{p, q} = \sum^S_{i = 1}\left(\|\mathbf{x}_i\|_p^p\right)^q$$.

In [7]:
for p in [1, 2, 4]:
    for q in [1, 2, 3]:
        print(f'||p norm where p = {p}, q = {q}')
        if p == 1 and q == 2:
            proximal.evaluate(proximal.GS_prox, proximal.GS_plus_nonneg_prox, n=10, size=(30, 30,), mean=0, sigma=1, lamb=0.5, additional_params=[(5, 5), p, q, True], err_thr=1e-3, plot=True)
            continue
        proximal.evaluate(proximal.GS_prox, proximal.GS_plus_nonneg_prox, n=10, size=(30, 30,), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[(5, 5), p, q, True], err_thr = 1e-3)

||p norm where p = 1, q = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.970e-04
Average absolute error: 2.771e-06

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 8.987e-05
Average absolute error: 1.096e-07

||p norm where p = 1, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 4.954e-01
Average absolute error: 7.704e-04

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.398e-04
Average absolute error: 1.858e-07

Plotting example as image.


Button(description='Show Widgets', style=ButtonStyle())

||p norm where p = 1, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 7.028e-01
Average absolute error: 7.641e-04

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 9.788e-05
Average absolute error: 1.622e-07

||p norm where p = 2, q = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.241e-04
Average absolute error: 5.856e-07

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.253e-04
Average absolute error: 4.502e-07

||p norm where p = 2, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.554e-01
Average absolute error: 8.129e-04

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 3.497e-04
Average absolute error: 2.908e-07

||p norm where p = 2, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.529e-01
Average absolute error: 1.005e-03

prox_reg(prox_no

## 4. Total Variation
[Back to Index](#Index)

The finite differences operator is denoted by $\operatorname{D}_n$, and it computes the gradient along the direction specified by $n$. In $1$-dimensional signals, there is no need to specify $n$.


## 4.A 1D Total Variation 
[Back to Index](#Index)

$$\|\operatorname{D}\mathbf{x}\|_p$$


In [8]:
for p in [1]:#, 2, 3, 5, 8, 'inf']:
    print(f'||p norm where p = {p}')
    if p == 1:
        proximal.evaluate(proximal.TV_1D_Lp_prox, proximal.TV_1D_Lp_plus_nonneg_prox, n=100, size=(30, ), mean=0, sigma=1, lamb=0.5, additional_params=[p], err_thr=1e-4, plot=True)
        continue
    proximal.evaluate(proximal.TV_1D_Lp_prox, proximal.TV_1D_Lp_plus_nonneg_prox, n=100, size=(30, ), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p], err_thr = 1e-4)

||p norm where p = 1
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.670e-04
Average absolute error: 3.794e-06

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.000e+00
Average absolute error: 1.568e-01

Plotting example


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

$$\|\operatorname{D}\mathbf{x}\|_p^p$$

In [9]:
for p in [1, 2, 3, 5, 8, 'inf']:
    print(f'||p^p norm where p = {p}')
    if p == 3:
        proximal.evaluate(proximal.TV_1D_Lp_prox, proximal.TV_1D_Lp_plus_nonneg_prox, n=10, size=(50, ), mean=0, sigma=1, lamb=0.5, additional_params=[p, True], err_thr=1e-4, plot=True)
        continue
    proximal.evaluate(proximal.TV_1D_Lp_prox, proximal.TV_1D_Lp_plus_nonneg_prox, n=10, size=(50, ), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p, True], err_thr = 1e-4)

||p^p norm where p = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 7.167e-05
Average absolute error: 3.156e-06

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.167e-01
Average absolute error: 1.700e-01

||p^p norm where p = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.486e-01
Average absolute error: 5.131e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 5.118e-01
Average absolute error: 1.246e-01

||p^p norm where p = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 5.425e-01
Average absolute error: 6.605e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 7.927e-01
Average absolute error: 1.318e-01

Plotting example


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p^p norm where p = 5
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 8.261e-01
Average absolute error: 7.661e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.574e-01
Average absolute error: 9.071e-02

||p^p norm where p = 8
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 7.771e-01
Average absolute error: 8.015e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 5.296e-01
Average absolute error: 7.822e-02

||p^p norm where p = inf


TypeError: '>' not supported between instances of 'str' and 'int'

### 4.B 2D Total Variation 
[Back to Index](#Index)

In the following cell I will test for $\operatorname{TV}$ with $\|\cdot\|_{p, q}$ mixed-norm. This includes nonisotropic $\operatorname{TV}$
$$\mathcal{R}(\mathbf{x}) = \sum^N_{n = 1}\|\mathbf{D}_1\mathbf{x}\|_n + \|\mathbf{D}_2\mathbf{x}\|_n$$ 
for $p=q=1$, and isotropic $\operatorname{TV}$ 
$$\mathcal{R}(\mathbf{x}) = \sum^N_{n = 1}\sqrt{[\mathbf{D_1x}]^2_n + [\mathbf{D_2x}]^2_n}$$
for $p=2$ and $q=1$.

In [ ]:
for p in [1, 2]:
    for q in [1, 2, 3]:
        print(f'|TV*|p,q norm where p = {p} and q = {q}.')
        if p == 2 and q == 2:
            proximal.evaluate(proximal.TV_prox, proximal.TV_plus_nonneg_prox, n=10, size=(40, 40), mean=0, sigma=1, lamb=0.5, additional_params=[p, q, False], err_thr=1e-4, plot=True)
            continue
        proximal.evaluate(proximal.TV_prox, proximal.TV_plus_nonneg_prox, n=10, size=(40, 40), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p, q, False], err_thr = 1e-3)

The following cell will test for $\operatorname{TV}$ with $\|\cdot\|_{p, q}^{p, q}$ mixed-norm.

In [ ]:
for p in [1, 2]:
    for q in [1, 2]:
        print(f'|TV*|p,q norm where p = {p} and q = {q}.')
        if p == 2 and q == 2:
            proximal.evaluate(proximal.TV_prox, proximal.TV_plus_nonneg_prox, n=40, size=(30, 30), mean=0, sigma=1, lamb=0.5, additional_params=[p, q, True], err_thr=1e-4, plot=True)
            continue
        proximal.evaluate(proximal.TV_prox, proximal.TV_plus_nonneg_prox, n=40, size=(30, 30), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p, q, True], err_thr = 1e-3)

## 5. Schatten Norm
[Back to Index](#Index)

This section will test for the Schatten norm, defined by
$$\|\mathbf{X}\|_{S_p} = \left( \sum_{k=1}^{\min\left(n_{1},n_{2}\right)}\mathbf{\sigma}_{k}^{p}\left({\mathbf X}\right)\right)^\frac{1}{p}$$, 
where $\mathbf{\sigma}_{k}^{p}\left({\bf X}\right)$ denotes the $k_{\mathrm{th}}$ singular value of $\bf{X}$

The values tested will be  $p\in\{1, 2, +\infty\}$.

In [ ]:
for p in ['nuc', 'fro', 2]:
    print(f'||p norm where p = {p}')
    if p == 'nuc':
        proximal.evaluate(proximal.schatten_prox, proximal.schatten_plus_nonneg_prox, n=100, size=(2, 2), mean=0, sigma=100, lamb=0.1, additional_params=[p], err_thr = 1e-4, plot=True)
        continue
    else:
        proximal.evaluate(proximal.schatten_prox, proximal.schatten_plus_nonneg_prox, n=100, size=(2, 2), mean=0, sigma=100, lamb=0.1, additional_params=[p], err_thr = 1e-4)

## 6. Hessian Schatten Norm
[Back to Index](#Index)

in the following cell I will test for the Hessian-Schatten norm, defined by
$$\mathcal{R}(x) = \sum^N_{n = 1}\| \left[
  \begin{bmatrix}
    [D_{11}x]_n [D_{12}x]_n\\
    [D_{21}x]_n [D_{22}x]_n
  \end{bmatrix}
\right] \|_{S_p}$$
with the Schatten norm taken over $p\in\{1, 2, +\infty\}$

In [ ]:
for p in ['nuc', 'fro', 2]:
    print(f'Hessian-Schatten norm:')
    proximal.evaluate(proximal.Hessian_Schatten_prox, proximal.Hessian_Schatten_plus_nonneg_prox, 
                  n=10, size=(15, 15), mean=0, sigma=10, lamb=0.5, err_thr = 1e-4, plot=True, additional_params = [p])

## References

[1.](https://arc.aiaa.org/doi/pdf/10.2514/1.26320?casa_token=EpyiTfodqo4AAAAA%3AGmnFHyrImbNxMMk2ONs1c9wpN6bLTip8_a7irQweswoms0vMBtL1kGu6h8v6IK76_zhhhtk0KPA&) Pol del Aguila Pla and Joakim Jaldén, Cell detection by functional inverse diffusion and non-negative group sparsity—Part II: Proximal optimization and Performance evaluation, IEEE Transactions on Signal Processing, vol. 66, no. 20, pp. 5422–5437, 2018

[2.](http://bigwww.epfl.ch/publications/soubies1904.pdf) E. Soubies, F. Soulez, M.T. McCann, T.-a. Pham, L. Donati, T. Debarre, D. Sage, M. Unser, "Pocket Guide to Solve Inverse Problems with GlobalBioIm," Inverse Problems, vol. 35, no. 10, paper no. 104006, pp. 1-20, October 2019